In [1]:
import re
import pandas as pd

In [2]:
text='[Tue Nov 29 21:33:23 2005] [error] [client 148.233.91.2] script not found or unable to stat: /var/www/cgi-bin/ip.cgi'

In [3]:
h,m=re.match(r'(.*\])(.*)',text).groups()

In [5]:
[token for token in re.split(r'(\[|\])|\s|((?<!\d\d)\:)',h) if token!='' and token!=None]

['[',
 'Tue',
 'Nov',
 '29',
 '21:33:23',
 '2005',
 ']',
 '[',
 'error',
 ']',
 '[',
 'client',
 '148.233.91.2',
 ']']

In [6]:
[token for token in re.split(r'(\[|\])|\s|((?<!\d\d)\:)',m) if token!='' and token!=None]

['script',
 'not',
 'found',
 'or',
 'unable',
 'to',
 'stat',
 ':',
 '/var/www/cgi-bin/ip.cgi']

In [7]:
f=open('./apache_15k.log')
lines=f.readlines()

In [8]:
header=[]
msg=[]
for line in lines:
    h,m=re.match(r'(.*\])(.*)',line).groups()
    header.append(h.strip())
    msg.append(m.strip())

In [9]:
header_tokens=[]
for h in header:
    tokens=[token for token in re.split(r'(\[|\])|\s|((?<!\d\d)\:)',h) if token!='' and token!=None]
    header_tokens.append(tokens)

In [10]:
msg_tokens=[]
for m in msg:
    tokens=[token for token in re.split(r'\[|\]|\s|((?<!\d\d)\:(?!\/\/))',m) if token!='' and token!=None]
    msg_tokens.append(tokens)

In [12]:
df=pd.DataFrame({'header':header,'msg':msg,'header_tokens':header_tokens,'msg_tokens':msg_tokens})

In [14]:
df.to_csv('./apache_parsed.csv')

In [15]:
header_anno=[]
for tokens in header_tokens:
    anno=[[token,'<O>'] for token in tokens]
    anno[1][1],anno[2][1],anno[3][1],anno[4][1],anno[8][1]='<DATE>','<DATE>','<DATE>','<TIME>','<LVL>'

    header_anno.append(anno)

In [16]:
header_anno[0]

[['[', '<O>'],
 ['Fri', '<DATE>'],
 ['Feb', '<DATE>'],
 ['24', '<DATE>'],
 ['17:07:21', '<TIME>'],
 ['2006', '<O>'],
 [']', '<O>'],
 ['[', '<O>'],
 ['error', '<LVL>'],
 [']', '<O>'],
 ['[', '<O>'],
 ['client', '<O>'],
 ['85.214.20.161', '<O>'],
 [']', '<O>']]

In [17]:
def sub_tag(series, patt_tag_list):
    def _sub_tag(series, patt, tag):
        for line in series:
            for pair in line:
                if re.search(patt,pair[0]):
                    pair[1]=tag
    for patt,tag in patt_tag_list:
        _sub_tag(series,patt,tag)

In [18]:
msg_anno=[]
for tokens in msg_tokens:
    anno=[[token,'<O>'] for token in tokens]

    msg_anno.append(anno)

In [19]:
re_date=r'\d{2}\/\d{2}\/\d{4}'
re_time=r'^\d{2}:\d{2}$'

re_obj=r'[a-zA-Z]+-\d{1,3}$'
re_cls=r'(\.([A-Z]+[a-z]*)+)|([A-Z]+[a-z]+){2,}'
re_func=r'\S*\(\)$'
re_path=r'^\'?\/'

re_host=r':\d{3,5}$'
re_url=r'[a-z]+:\/\/\S*'


In [20]:
sub_tag(msg_anno,
    [(re_cls,'<CLS>'),
    (re_func,'<FUNC>'),
    (re_path,'<PATH>'),
    (re_host,'<HOST>'),
    (re_url,'<URL>')])

In [22]:
df=pd.DataFrame({
    'msg_anno':msg_anno,
    'header_anno':header_anno,
    'header':header,
    'msg':msg,
    'header_tokens':header_tokens,
    'msg_tokens':msg_tokens})
df.to_csv('./apache_parsed.csv')

In [23]:
for line in df.msg_anno[10:20]:
    print(str(line)+'\n')

[['File', '<O>'], ['does', '<O>'], ['not', '<O>'], ['exist', '<O>'], [':', '<O>'], ['/var/www/html/blog', '<PATH>']]

[['File', '<O>'], ['does', '<O>'], ['not', '<O>'], ['exist', '<O>'], [':', '<O>'], ['/var/www/html/stat-cgi', '<PATH>']]

[['mod_jk', '<O>'], ['child', '<O>'], ['init', '<O>'], ['1', '<O>'], ['-2', '<O>']]

[['workerEnv.init()', '<FUNC>'], ['ok', '<O>'], ['/etc/httpd/conf/workers2.properties', '<PATH>']]

[['File', '<O>'], ['does', '<O>'], ['not', '<O>'], ['exist', '<O>'], [':', '<O>'], ['/var/www/html/cache', '<PATH>']]

[['mod_jk2', '<O>'], ['Shutting', '<O>'], ['down', '<O>']]

[['File', '<O>'], ['does', '<O>'], ['not', '<O>'], ['exist', '<O>'], [':', '<O>'], ['/var/www/html/cvs', '<PATH>']]

[['File', '<O>'], ['does', '<O>'], ['not', '<O>'], ['exist', '<O>'], [':', '<O>'], ['/var/www/html/blog', '<PATH>']]

[['jk2_init()', '<FUNC>'], ['Found', '<O>'], ['child', '<O>'], ['6968', '<O>'], ['in', '<O>'], ['scoreboard', '<O>'], ['slot', '<O>'], ['1', '<O>']]

[['jk2_init